In [0]:
!pip3 install pyfasttext

     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
from pyfasttext import FastText
import numpy as np

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0, 'drive/My Drive/WordEmbed workshop/')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from utils import *
from w2v_utils import *

Using TensorFlow backend.


# Operations on word vectors

Welcome to your first activity. 

Because word embeddings are very computionally expensive to train, most ML practitioners will load a pre-trained set of embeddings. 

**After this assignment you will be able to:**

- Load pre-trained word vectors, and measure similarity using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______.  

Let's get started! Run the following cell to load pre-trained word vectors (would take a couple of minutes to load)

In [4]:
embeddings_index = FastText("drive/My Drive/WordEmbed workshop/cc.en.300.bin") 

NameError: ignored

In [0]:
#play with different index values
list(embeddings_index.words)[:10], embeddings_index['the'],len( embeddings_index['the'])

([',', 'the', '.', 'and', 'to', 'of', 'a', '</s>', 'in', 'is'],
 array('f', [-0.05174419283866882, 0.0739639550447464, -0.013056879863142967, 0.04472655802965164, -0.03432036563754082, 0.02121688425540924, 0.006911486387252808, -0.01632784679532051, -0.01807485707104206, -0.0019965237006545067, -0.10204669088125229, 0.005904885940253735, 0.025654055178165436, -0.002596620935946703, -0.05855605751276016, -0.037758685648441315, 0.016311872750520706, 0.014632370322942734, -0.008759298361837864, -0.01759478449821472, -0.008547326549887657, -0.007793376222252846, -0.018278032541275024, 0.008798242546617985, 0.0013020262122154236, -0.09382941573858261, 0.013899145647883415, 0.014892999082803726, -0.03937097638845444, -0.029441121965646744, 0.009422930888831615, -0.02522841840982437, -0.010441077873110771, -0.22131945192813873, -0.022859765216708183, -0.008935268968343735, -0.032222650945186615, 0.08217015862464905, 0.0020999780390411615, 0.028173504397273064, 0.007170667871832848, -0.0091256

# 1 - Cosine similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

<img src="https://drive.google.com/uc?id=16VOhhx1BuX5bPpWAH_L5mRbCS3IqJCwf">
<caption><center> **Figure 1**: The cosine of the angle between two vectors is a measure of how similar they are</center></caption>

The function `cosine_similarity()` is implemented for you.

In [0]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u, v)
    # Compute the L2 norm of u (≈1 line)
    norm_u =np.linalg.norm(u)
#     norm_u = np.sqrt(np.sum(u * u))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.linalg.norm(v)
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot / (norm_u * norm_v)
        
    return cosine_similarity

In [0]:
# some similarity values are hard-coded here. You can change them and try your own ones
father = embeddings_index["father"]
mother = embeddings_index["mother"]
ball = embeddings_index["ball"]
crocodile = embeddings_index["crocodile"]
france = embeddings_index["france"]
italy = embeddings_index["italy"]
paris = embeddings_index["paris"]
rome = embeddings_index["rome"]


print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = "
      ,cosine_similarity(np.subtract(france, paris), np.subtract(rome, italy)))

king = embeddings_index["king"]
queen = embeddings_index["queen"]
man = embeddings_index["man"]
woman = embeddings_index["woman"]

print("cosine_similarity(france - paris, rome - italy) = "
      ,cosine_similarity(np.subtract(king, man), np.subtract(queen, woman)))

raven = embeddings_index["raven"]
nevermore = embeddings_index["nevermore"]
print("cosine_similarity(raven, nevermore) = ", cosine_similarity(raven, nevermore))

pig = embeddings_index["pig"]
oink = embeddings_index["oink"]
print("cosine_similarity(pig, oink) = ", cosine_similarity(pig, oink))

print("cosine_similarity(oink - pig, nevermore - raven) = "
      ,cosine_similarity(np.subtract(oink, pig), np.subtract(nevermore, raven)))


cosine_similarity(father, mother) =  0.83168906
cosine_similarity(ball, crocodile) =  0.16411223
cosine_similarity(france - paris, rome - italy) =  -0.26101252
cosine_similarity(france - paris, rome - italy) =  0.75547177
cosine_similarity(raven, nevermore) =  0.5220859
cosine_similarity(pig, oink) =  0.57947254
cosine_similarity(oink - pig, nevermore - raven) =  0.29101586


## 2 - Word analogy task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 

In [0]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    
#     words = word_to_vec_map.words
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for w in word_to_vec_map.words:        
        # to avoid best_word being one of the input words, pass on them.
        if w in [word_a, word_b, word_c] :
            continue
        
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity(np.subtract(e_b, e_a),
                                       np.subtract(word_to_vec_map[w], e_c))
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w        
        
    return best_word

Run the cell below to test your code, this may take 1-2 minutes.

In [0]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad, embeddings_index)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> delhi.
man -> woman :: boy -> girl
small -> smaller :: large -> larger


**Expected Output**:

<table>
    <tr>
        <td>
            **italy -> italian** ::
        </td>
        <td>
         spain -> spanish
        </td>
    </tr>
        <tr>
        <td>
            **india -> delhi** ::
        </td>
        <td>
         japan -> tokyo
        </td>
    </tr>
        <tr>
        <td>
            **man -> woman ** ::
        </td>
        <td>
         boy -> girl
        </td>
    </tr>
        <tr>
        <td>
            **small -> smaller ** ::
        </td>
        <td>
         large -> larger
        </td>
    </tr>
</table>

Once you get the correct expected output, please feel free to modify the input cells above to test your own analogies. Try to find some other analogy pairs that do work, but also find some where the algorithm doesn't give the right answer: For example, you can try small->smaller as big->?.  

### Congratulations!

You've come to the end of this task. Here are the main points you should remember:

- Cosine similarity a good way to compare similarity between pairs of word vectors. (Though L2 distance works too.) 
- For NLP applications, using a pre-trained set of word vectors from the internet is often a good way to get started. 